In [ ]:
import pandas as pd
import numpy as np

path = 'E:/BaiduYunDownload/DC_2/'
data = pd.read_csv(path+'data0219.csv',encoding = 'gb18030',index_col=None)
data = data.fillna(0)

for index,i in enumerate(data.columns):
    if 'cost_sum' in i:
        data[i+str(index)] = data[i]/data['costpos_sum']
for index,i in enumerate(data.columns):
    if 'count' in i:
        data[i+str(index)] = data[i]/data['costpos_count']

In [ ]:
from sklearn import metrics
from sklearn import cross_validation
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from imblearn.over_sampling import RandomOverSampler

data = data.fillna(0)
train_data = data[data['TT']=='train']
x = train_data.drop(['TT','student_id','grant_amount','class_rank',],axis = 1)
y = train_data['grant_amount']

test_data = data[data['TT']=='test']
test_x = test_data.drop(['TT','student_id','grant_amount','class_rank',],axis = 1)
test_y = test_data['grant_amount']

os =  RandomOverSampler(ratio=1.0)
X_overs, y_overs = os.fit_sample(x, y)


data_train, data_test, target_train, target_test = cross_validation.train_test_split(X_overs, y_overs)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y_overs, n_folds=5, shuffle=True)

new_train = np.zeros((X_overs.shape[0],4))
new_test = np.zeros((test_x.shape[0],4))

for i,(trainid,valid) in enumerate(skf):
    print 'fold' + str(i)
    train_x = X_overs[trainid]
    train_y = y_overs[trainid]
    val_x = X_overs[valid]
    model = XGBClassifier(learning_rate =0.02 ,n_estimators=800,max_depth=3,min_child_weight=0.5,gamma = 0.01,subsample=0.7,
                          colsample_bytree=0.7,colsample_bylevel=0.6,objective= 'multi:softmax',seed = 10,nthread=8)
    model.fit(train_x, train_y, eval_metric='auc')
    new_train[valid] = model.predict_proba(val_x)
    new_test += model.predict_proba(test_x)
    
new_test /= 5
stacks = []
stacks_name = []
stack = np.vstack([new_train,new_test])
stacks.append(stack)
stacks_name += ['%s_%d'%('xgb_',i) for i in range(4)]
stacks = np.hstack(stacks)
xgb_stacks = pd.DataFrame(data=stacks,columns=stacks_name)
xgb_stacks.to_csv(path+'xgb.csv',sep = ',', index=False)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y_overs, n_folds=5, shuffle=True)

new_train = np.zeros((X_overs.shape[0],4))
new_test = np.zeros((test_x.shape[0],4))

for i,(trainid,valid) in enumerate(skf):
    print 'fold' + str(i)
    train_x = X_overs[trainid]
    train_y = y_overs[trainid]
    val_x = X_overs[valid]
    model = RandomForestClassifier(n_estimators=500,bootstrap=True,min_samples_leaf=11,max_depth=15,n_jobs=-1,
                       class_weight='balanced_subsample',max_features='sqrt',criterion='gini')
    model.fit(train_x, train_y)
    new_train[valid] = model.predict_proba(val_x)
    new_test += model.predict_proba(test_x)
    
new_test /= 5
stacks = []
stacks_name = []
stack = np.vstack([new_train,new_test])
stacks.append(stack)
stacks_name += ['%s_%d'%('rf_gini_',i) for i in range(4)]
stacks = np.hstack(stacks)
RF_gini_stacks = pd.DataFrame(data=stacks,columns=stacks_name)
RF_gini_stacks.to_csv(path+'rf_gini.csv',sep = ',', index=False)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y_overs, n_folds=5, shuffle=True)

new_train = np.zeros((X_overs.shape[0],4))
new_test = np.zeros((test_x.shape[0],4))

for i,(trainid,valid) in enumerate(skf):
    print 'fold' + str(i)
    train_x = X_overs[trainid]
    train_y = y_overs[trainid]
    val_x = X_overs[valid]
    model = RandomForestClassifier(n_estimators=500,bootstrap=True,min_samples_leaf=11,max_depth=15,n_jobs=-1,
                       class_weight='balanced_subsample',max_features='sqrt',criterion='entropy')
    model.fit(train_x, train_y,)
    new_train[valid] = model.predict_proba(val_x)
    new_test += model.predict_proba(test_x)
    
new_test /= 5
stacks = []
stacks_name = []
stack = np.vstack([new_train,new_test])
stacks.append(stack)
stacks_name += ['%s_%d'%('rf_en_',i) for i in range(4)]
stacks = np.hstack(stacks)
RF_en_stacks = pd.DataFrame(data=stacks,columns=stacks_name)
RF_en_stacks.to_csv(path+'rf_en.csv',sep = ',', index=False)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y_overs, n_folds=5, shuffle=True)

new_train = np.zeros((X_overs.shape[0],4))
new_test = np.zeros((test_x.shape[0],4))

for i,(trainid,valid) in enumerate(skf):
    print 'fold' + str(i)
    train_x = X_overs[trainid]
    train_y = y_overs[trainid]
    val_x = X_overs[valid]
    model = ExtraTreesClassifier(n_estimators=500,bootstrap=True,min_samples_leaf=11,max_depth=15,n_jobs=-1,
                       class_weight='balanced_subsample',max_features='sqrt',criterion='gini')
    model.fit(train_x, train_y,)
    new_train[valid] = model.predict_proba(val_x)
    new_test += model.predict_proba(test_x)
    
new_test /= 5
stacks = []
stacks_name = []
stack = np.vstack([new_train,new_test])
stacks.append(stack)
stacks_name += ['%s_%d'%('etc_gini_',i) for i in range(4)]
stacks = np.hstack(stacks)
ETC_gini_stacks = pd.DataFrame(data=stacks,columns=stacks_name)
ETC_gini_stacks.to_csv(path+'etc_gini.csv',sep = ',', index=False)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y_overs, n_folds=5, shuffle=True)

new_train = np.zeros((X_overs.shape[0],4))
new_test = np.zeros((test_x.shape[0],4))

for i,(trainid,valid) in enumerate(skf):
    print 'fold' + str(i)
    train_x = X_overs[trainid]
    train_y = y_overs[trainid]
    val_x = X_overs[valid]
    model = ExtraTreesClassifier(n_estimators=500,bootstrap=True,min_samples_leaf=11,max_depth=15,n_jobs=-1,
                       class_weight='balanced_subsample',max_features='sqrt',criterion='entropy')
    model.fit(train_x, train_y,)
    new_train[valid] = model.predict_proba(val_x)
    new_test += model.predict_proba(test_x)
    
new_test /= 5
stacks = []
stacks_name = []
stack = np.vstack([new_train,new_test])
stacks.append(stack)
stacks_name += ['%s_%d'%('etc_en_',i) for i in range(4)]
stacks = np.hstack(stacks)
ETC_en_stacks = pd.DataFrame(data=stacks,columns=stacks_name)
ETC_en_stacks.to_csv(path+'etc_en.csv',sep = ',', index=False)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y_overs, n_folds=3, shuffle=True)

new_train = np.zeros((X_overs.shape[0],4))
new_test = np.zeros((test_x.shape[0],4))

for i,(trainid,valid) in enumerate(skf):
    print 'fold' + str(i)
    train_x = X_overs[trainid]
    train_y = y_overs[trainid]
    val_x = X_overs[valid]
    model = GradientBoostingClassifier(learning_rate=0.02,n_estimators=400,subsample = 0.8,random_state = 10)
    model.fit(train_x, train_y,)
    new_train[valid] = model.predict_proba(val_x)
    new_test += model.predict_proba(test_x)
    
new_test /= 3
stacks = []
stacks_name = []
stack = np.vstack([new_train,new_test])
stacks.append(stack)
stacks_name += ['%s_%d'%('GBDT_',i) for i in range(4)]
stacks = np.hstack(stacks)
GBDT_stacks = pd.DataFrame(data=stacks,columns=stacks_name)
GBDT_stacks.to_csv(path+'GBDT.csv',sep = ',', index=False)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y_overs, n_folds=5, shuffle=True)

new_train = np.zeros((X_overs.shape[0],4))
new_test = np.zeros((test_x.shape[0],4))

for i,(trainid,valid) in enumerate(skf):
    print 'fold' + str(i)
    train_x = X_overs[trainid]
    train_y = y_overs[trainid]
    val_x = X_overs[valid]
    model = BaggingClassifier(n_estimators=400)
    model.fit(train_x, train_y,)
    new_train[valid] = model.predict_proba(val_x)
    new_test += model.predict_proba(test_x)
    
new_test /= 5
stacks = []
stacks_name = []
stack = np.vstack([new_train,new_test])
stacks.append(stack)
stacks_name += ['%s_%d'%('bag_',i) for i in range(4)]
stacks = np.hstack(stacks)
bag_stacks = pd.DataFrame(data=stacks,columns=stacks_name)
bag_stacks.to_csv(path+'bag.csv',sep = ',', index=False)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y_overs, n_folds=5, shuffle=True)

new_train = np.zeros((X_overs.shape[0],4))
new_test = np.zeros((test_x.shape[0],4))

for i,(trainid,valid) in enumerate(skf):
    print 'fold' + str(i)
    train_x = X_overs[trainid]
    train_y = y_overs[trainid]
    val_x = X_overs[valid]
    model = AdaBoostClassifier(n_estimators=400,learning_rate=0.02)
    model.fit(train_x, train_y,)
    new_train[valid] = model.predict_proba(val_x)
    new_test += model.predict_proba(test_x)
    
new_test /= 5
stacks = []
stacks_name = []
stack = np.vstack([new_train,new_test])
stacks.append(stack)
stacks_name += ['%s_%d'%('ada_',i) for i in range(4)]
stacks = np.hstack(stacks)
ada_stacks = pd.DataFrame(data=stacks,columns=stacks_name)
ada_stacks.to_csv(path+'ada.csv',sep = ',', index=False)